In [68]:
!uv pip install duckdb==1.4.3 python-dotenv numpy pandas

Using Python 3.11.14 environment at: /Users/bgramaje/workspace/MUCEIM/bigdata/muceim-airflow/.venv
Audited 4 packages in 3ms


In [69]:
import sys
from pathlib import Path
from dotenv import load_dotenv

current_dir = Path.cwd()
dags_path = current_dir.parent / 'dags'

dags_path_str = str(dags_path.absolute())
if dags_path_str not in sys.path:
    sys.path.insert(0, dags_path_str)

load_dotenv(dotenv_path=str('../.env'), override=True)

from utils.utils import get_ducklake_connection

print(f"✅ Variables de entorno cargadas")
print(f"✅ Path agregado: {dags_path_str}")
print("✅ Connection manager importado")


✅ Variables de entorno cargadas
✅ Path agregado: /Users/bgramaje/workspace/MUCEIM/bigdata/muceim-airflow/dags
✅ Connection manager importado


In [70]:
# Obtener conexión DuckLake usando el connection manager
import duckdb
print(f"🦆 DuckDB version: {duckdb.__version__}")

con = get_ducklake_connection()

print("✅ Conexión DuckLake establecida")
print("✅ Todas las extensiones cargadas automáticamente")
print("✅ DuckLake attachado y listo para usar")


🦆 DuckDB version: 1.4.3
✅ Conexión DuckLake establecida
✅ Todas las extensiones cargadas automáticamente
✅ DuckLake attachado y listo para usar


In [71]:
# Wrapper SQL para simplificar consultas y liberar memoria
import gc

class SQL:
    """Wrapper para ejecutar queries SQL de forma simple y liberar memoria"""
    def __init__(self, connection):
        self.con = connection
    
    def __call__(self, query):
        """Ejecuta una query y retorna el resultado como DataFrame (se muestra automáticamente en notebooks).
        Libera memoria de DuckDB después de ejecutar la query."""
        result = self.con.execute(query).fetchdf()
        # Liberar memoria de DuckDB después de fetchdf
        try:
            # Forzar liberación de memoria de DuckDB
            self.con.execute("PRAGMA force_checkpoint")
        except:
            pass

        gc.collect()
        return result
    
    def fetch_df(self, query):
        """Ejecuta una query y retorna un DataFrame"""
        result = self.con.execute(query).fetchdf()
        try:
            self.con.execute("PRAGMA force_checkpoint")
        except:
            pass
        return result

# Crear instancia global SQL
SQL = SQL(con)
print("✅ Wrapper SQL creado. Usa SQL(query) para ejecutar consultas - el DataFrame se mostrará automáticamente")
print("✅ El wrapper libera memoria automáticamente después de cada consulta")

✅ Wrapper SQL creado. Usa SQL(query) para ejecutar consultas - el DataFrame se mostrará automáticamente
✅ El wrapper libera memoria automáticamente después de cada consulta


In [72]:
SQL("""
    SELECT * 
    FROM (
        SELECT * 
        FROM bronze_mitma_od_municipios
        WHERE fecha >= '2023-06-01' AND fecha <= '2023-06-02'
    ) AS filtered_data
    USING SAMPLE 10;
""")

,fecha,periodo,origen,destino,distancia,actividad_origen,actividad_destino,estudio_origen_posible,estudio_destino_posible,residencia,renta,edad,sexo,viajes,viajes_km,loaded_at,source_file
0,2023-06-01,00,18022,14021,>50,casa,no_frecuente,no,no,18,<10,0-25,NA,6.816,823.661,2026-01-12 12:27:40.160799+01:00,https://movilidad-opendata.mitma.es/estudios_b...
1,2023-06-01,00,19130,19105,2-10,frecuente,frecuente,no,no,19,10-15,NA,NA,3.117,9.696,2026-01-12 12:27:40.160799+01:00,https://movilidad-opendata.mitma.es/estudios_b...
2,2023-06-01,00,20042,20022_AM,2-10,no_frecuente,casa,no,no,20,>15,NA,NA,2.974,22.635,2026-01-12 12:27:40.160799+01:00,https://movilidad-opendata.mitma.es/estudios_b...
3,2023-06-01,00,03063,03082,2-10,frecuente,frecuente,no,no,03,10-15,65-100,NA,3.087,21.798,2026-01-12 12:27:40.160799+01:00,https://movilidad-opendata.mitma.es/estudios_b...
4,2023-06-01,00,03121,03065,2-10,frecuente,casa,no,no,03,10-15,45-65,mujer,16.051,82.583,2026-01-12 12:27:40.160799+01:00,https://movilidad-opendata.mitma.es/estudios_b...
5,2023-06-01,01,08089,08056,0.5-2,trabajo_estudio,casa,no,no,08,10-15,25-45,NA,4.753,4.11,2026-01-12 12:27:40.160799+01:00,https://movilidad-opendata.mitma.es/estudios_b...
6,2023-06-01,01,08112,08298,2-10,casa,no_frecuente,no,no,08,10-15,NA,NA,7.224,55.829,2026-01-12 12:27:40.160799+01:00,https://movilidad-opendata.mitma.es/estudios_b...
7,2023-06-01,03,08019,08019,2-10,trabajo_estudio,frecuente,no,no,08,10-15,45-65,hombre,2.122,4.339,2026-01-12 12:27:40.160799+01:00,https://movilidad-opendata.mitma.es/estudios_b...
8,2023-06-01,03,08097_AM,08259,10-50,frecuente,casa,no,no,08,10-15,NA,NA,9.004,119.206,2026-01-12 12:27:40.160799+01:00,https://movilidad-opendata.mitma.es/estudios_b...
9,2023-06-01,03,08114,08200,10-50,frecuente,casa,no,no,08,10-15,0-25,NA,4.788,92.493,2026-01-12 12:27:40.160799+01:00,https://movilidad-opendata.mitma.es/estudios_b...


In [73]:
SQL("""
    SELECT *
    FROM bronze_ine_empresas_municipio
    WHERE Nombre ILIKE '%Alcoi%' AND Nombre ILIKE '%CNAE%'
""")

,COD,Nombre,FK_Unidad,FK_Escala,Data,loaded_at,source_url,year
0,DIR415250,Alcoi/Alcoy. Total. Total de empresas. Total C...,97,1,"[{'Fecha': 1672527600000, 'FK_TipoDato': 1, 'F...",2026-01-12 15:25:35.531570+01:00,https://servicios.ine.es/wstempus/js/ES/DATOS_...,2023


In [74]:
# Filtrar tablas que comienzan con 'silver_' usando SQL wrapper
SQL("""
    SELECT table_name
    FROM information_schema.tables
    WHERE table_schema = 'main'
        AND table_name LIKE 'silver_%'
    ORDER BY table_name;
""")

,table_name
0,silver_ine_all
1,silver_mitma_distances
2,silver_mitma_ine_mapping
3,silver_mitma_od
4,silver_mitma_od_quality
5,silver_zones


In [ ]:
# silver_zones
SQL("""
    -- spatial extension needed to decode geometry_obj and centroid
    LOAD spatial;
    SELECT *,
    FROM silver_zones
""")

,id,nombre,geometry_obj,centroid
0,01054_AM,Urkabustaiz agregacion de municipios,"[3, 4, 0, 0, 0, 0, 0, 0, 24, 14, 59, 192, 73, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
1,02075_AM,Valdeganga agregacion de municipios,"[3, 4, 0, 0, 0, 0, 0, 0, 236, 108, 218, 191, 1...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
2,03002_AM,Agost agregacion de municipios,"[3, 4, 0, 0, 0, 0, 0, 0, 45, 11, 30, 191, 152,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
3,03009,Alcoy/Alcoi,"[3, 4, 0, 0, 0, 0, 0, 0, 33, 62, 4, 191, 168, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
4,03014,Alicante/Alacant,"[3, 4, 0, 0, 0, 0, 0, 0, 234, 177, 9, 191, 157...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
...,...,...,...,...
2564,50181_AM,Muel agregacion de municipios,"[3, 4, 0, 0, 0, 0, 0, 0, 74, 213, 122, 191, 11...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
2565,50034_AM,Ariza agregacion de municipios,"[3, 4, 0, 0, 0, 0, 0, 0, 120, 42, 250, 191, 16...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
2566,50225_AM,Ricla agregacion de municipios,"[3, 4, 0, 0, 0, 0, 0, 0, 218, 1, 195, 191, 174...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
2567,50223_AM,Remolinos agregacion de municipios,"[3, 4, 0, 0, 0, 0, 0, 0, 223, 196, 156, 191, 1...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."


In [89]:
SQL("""
    SELECT * 
    FROM (
        SELECT * 
        FROM silver_mitma_od
        WHERE fecha >= '2023-06-01' AND fecha <= '2023-06-02'
    ) AS filtered_data
    USING SAMPLE 10;
""")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,fecha,origen_zone_id,destino_zone_id,viajes,viajes_km,residencia,is_weekend,is_holiday
0,2023-06-02,43090_AM,43137_AM,21.250,178.189,43,False,False
1,2023-06-02,03011,03011,2.298,1.718,30,False,False
2,2023-06-02,17901_AM,17044,63.507,320.823,17,False,False
3,2023-06-02,08163,08203,105.683,611.233,08,False,False
4,2023-06-02,43007_AM,43043_AM,6.110,114.742,43,False,False
5,2023-06-02,08184,08182,2.319,60.423,08,False,False
6,2023-06-02,08085_AM,08305,1.799,4.933,14,False,False
7,2023-06-02,15078,15089,2.608,31.973,36,False,False
8,2023-06-02,36033,19245_AM,4.891,2604.104,19,False,False
9,2023-06-02,45168,45047_AM,7.158,215.305,45,False,False


In [91]:
SQL("""
    SELECT * FROM bronze_spanish_holidays 
    USING SAMPLE 10
""")

,date,name
0,2023-01-06,Epifanía del Señor
1,2023-04-07,Viernes Santo
2,2023-05-01,Día del Trabajador
3,2023-08-15,Asunción de la Virgen
4,2023-10-12,Fiesta Nacional de España
5,2023-11-01,Todos los Santos
6,2023-12-06,Día de la Constitución Española
7,2023-12-08,Inmaculada Concepción
8,2023-12-25,Navidad


In [28]:
SQL("""
    SELECT COUNT(*) FROM silver_mitma_od
    UNION ALL
    SELECT COUNT(*) FROM bronze_mitma_od_municipios
    UNION ALL
    SELECT COUNT(*) FROM silver_mitma_od_quality
""")

,count_star()
0,140825079
1,741162633
2,156340230


In [29]:
# silver_od_quality
SQL("SELECT * FROM silver_mitma_od_quality USING SAMPLE 10")


,fecha,origen_zone_id,destino_zone_id,residencia,viajes_per_capita,km_per_capita,flag_negative_viajes,flag_negative_viajes_km,z_viajes_per_capita,flag_outlier_viajes_per_capita
0,2023-04-11 07:00:00,29067,29094,29,0.001219,0.033488,False,False,-0.154245,False
1,2023-04-11 08:00:00,13034,41091,13,0.000031,0.007851,False,False,-0.292059,False
2,2023-04-11 11:00:00,39025,39060,39,0.003793,0.040828,False,False,0.144284,False
3,2023-04-11 21:00:00,25120,22225,25,0.000036,0.001049,False,False,-0.291550,False
4,2023-04-11 17:00:00,46207,46013,46,0.003418,0.034503,False,False,0.100818,False
5,2023-04-11 10:00:00,28059,28005,28,0.001006,0.018243,False,False,-0.178938,False
6,2023-04-11 16:00:00,48060_AM,48095_AM,48,0.003189,0.023270,False,False,0.074318,False
7,2023-04-11 16:00:00,11014,11007,29,0.000538,0.008891,False,False,-0.233316,False
8,2023-04-11 07:00:00,47129_AM,47010,47,0.000835,0.028870,False,False,-0.198837,False
9,2023-04-11 17:00:00,03092_AM,46200_AM,03,0.000185,0.001366,False,False,-0.274172,False


In [ ]:
# silver_overnight_stay
SQL("SELECT * FROM silver_overnight_stay USING SAMPLE 10")


,fecha,zona_pernoctacion,zona_residencia,personas
0,2023-03-02,28079,2702804_AD,53.827
1,2023-03-02,28145,28009,3.039
2,2023-03-02,41038,28014,14.980
3,2023-03-01,27005,27024_AM,7.306
4,2023-03-02,34056_AM,0105902,3.170
5,2023-03-02,16078,0200303,7.500
6,2023-03-02,49178_AM,4718610,12.549
7,2023-03-02,39082_AM,4802006,4.137
8,2023-03-01,27021_AM,1503006,9.185
9,2023-03-01,02003,23082_AM,22.500


In [ ]:
# silver_people_day
SQL("SELECT * FROM silver_people_day USING SAMPLE 10")


,fecha,zona_pernoctacion,edad,sexo,numero_viajes,personas
0,2023-03-02,06159,25-45,hombre,2+,155.203
1,2023-03-02,08136,65-100,mujer,1,48.698
2,2023-03-02,08282,0-25,hombre,1,27.397
3,2023-03-02,11015,65-100,mujer,1,184.052
4,2023-03-02,12031,65-100,mujer,1,12.456
5,2023-03-02,15009,65-100,hombre,2,302.083
6,2023-03-01,03064,65-100,hombre,0,264.871
7,2023-03-01,28168_AM,0-25,hombre,2,64.415
8,2023-03-01,22907_AM,65-100,hombre,2,130.622
9,2023-03-01,25204_AM,45-65,mujer,2,171.379


In [ ]:
# silver_mitma_distances
SQL("SELECT * FROM silver_mitma_distances USING SAMPLE 10")


,origin,destination,distance_km
0,02045,02073_AM,15.915524
1,02045,04029,293.576838
2,03043,05002_AM,475.104811
3,04064,05168_AM,473.471941
4,01051,14049,657.831267
5,03053,17164_AM,505.905016
6,12902_AM,17169_AM,378.949676
7,08097_AM,20022_AM,511.946709
8,08088,23050,797.580336
9,08125,23055,749.165117


In [ ]:
# silver_mitma_ine_mapping
SQL("SELECT * FROM silver_mitma_ine_mapping USING SAMPLE 10")


,nombre,codigo_ine,municipio_mitma
0,barbadillo de herreros,09037,09330_AM
1,bohonal de ibor,10030,10140_AM
2,fuentes de ayodar,12064,12072_AM
3,algar de mesa,19016,19086_AM
4,zufre,21079,21069_AM
5,garganta de los montes,28062,28027_AM
6,barillas,31048,31068_AM
7,malpartida,37178,37174_AM
8,"riba de escalote, la",42157,42035_AM
9,canizal,49034,49081_AM


In [ ]:
# silver_ine_all
SQL("SELECT * FROM silver_ine_all USING SAMPLE 10")


,id,nombre,empresas,renta_media,poblacion_total,poblacion_hombres,poblacion_mujeres,year
0,08068,Cervelló,803.0,18664.0,9743.0,4911.0,4832.0,
1,10116_AM,Malpartida de Plasencia agregacion de municipios,319.0,11008.0,5236.0,2608.0,2628.0,
2,13092_AM,Villanueva de la Fuente agregacion de municipios,402.0,11341.0,6142.0,3198.0,2944.0,
3,19143,Horche,126.0,13502.0,3119.0,1596.0,1523.0,
4,22907_AM,Aínsa-Sobrarbe agregacion de municipios,424.0,17618.0,5222.0,2712.0,2510.0,
5,28137,"Santos de la Humosa, Los",126.0,13849.0,2867.0,1486.0,1381.0,
6,37156,Guijuelo,525.0,13971.0,5427.0,2776.0,2651.0,
7,40906_AM,San Cristóbal de Segovia agregacion de municipios,388.0,14781.0,5796.0,2955.0,2841.0,
8,50099,Épila,219.0,12570.0,4704.0,2431.0,2273.0,
9,50219,"Puebla de Alfindén, La",708.0,14581.0,6550.0,3356.0,3194.0,


In [32]:
# silver_zones_all
SQL("SELECT * FROM silver_zones_all USING SAMPLE 10")


,id,nombre,geometry_obj,centroid
0,07001,Alaró,"[3, 4, 0, 0, 0, 0, 0, 0, 125, 90, 51, 64, 230,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
1,07033,Manacor,"[3, 4, 0, 0, 0, 0, 0, 0, 119, 17, 79, 64, 34, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
2,20036,Hondarribia,"[3, 4, 0, 0, 0, 0, 0, 0, 129, 180, 233, 191, 4...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
3,29090_AM,Tolox agregacion de municipios,"[3, 4, 0, 0, 0, 0, 0, 0, 55, 113, 157, 192, 14...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
4,08149_AM,Olost agregacion de municipios,"[3, 4, 0, 0, 0, 0, 0, 0, 194, 162, 6, 64, 131,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
5,36038,Pontevedra,"[3, 4, 0, 0, 0, 0, 0, 0, 235, 234, 9, 193, 170...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
6,45118_AM,Novés agregacion de municipios,"[3, 4, 0, 0, 0, 0, 0, 0, 199, 76, 136, 192, 16...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
7,15024,Cerceda,"[3, 4, 0, 0, 0, 0, 0, 0, 16, 189, 7, 193, 42, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
8,47165,Tordesillas,"[3, 4, 0, 0, 0, 0, 0, 0, 216, 47, 160, 192, 24...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
9,12027,Benicarló,"[3, 4, 0, 0, 0, 0, 0, 0, 170, 67, 206, 62, 111...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
